In [15]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import warnings
from astropy.wcs import WCS
from astropy.io import fits
from scipy.optimize import curve_fit, root_scalar
from spectral_cube import SpectralCube
from lmfit import Model, Parameters
from scipy.interpolate import interp1d
import pandas

In [16]:
data_path = '/home/scratch/hfwest/Pilot/'

nu11 = 23.6944955e9
original = SpectralCube.read(data_path + 'Data/Pilot_NH3_11_bl2.fits').with_spectral_unit(u.km/u.s, velocity_convention='radio', rest_value=nu11*u.Hz)
amp = fits.open(data_path + 'Results/Pilot_NH3_11_bl2_Max11_test.fits')[0].data
sigma = fits.open(data_path + 'Results/Pilot_NH3_11_bl2_Sigma11_test.fits')[0].data
vel = fits.open(data_path + 'Results/Pilot_NH3_11_bl2_Vel11_test.fits')[0].data

In [ ]:
nx = original.shape[2]
ny = original.shape[1]

x_range = original.spectral_axis.value

gaussian_data = np.zeros_like(original)

def gaussian(x, amp, mean, sigma):
    return amp * np.exp(-(x - mean)**2 / (2 * sigma**2))

In [18]:
for j in range(ny):
    for i in range(nx):
        gaussian_data[:, j, i] = gaussian(x_range, amp[j, i], vel[j, i], sigma[j, i])

/tmp/ipykernel_1324633/458745518.py:10: RuntimeWarning: divide by zero encountered in divide
  return amp * np.exp(-(x - mean)**2 / (2 * sigma**2))


In [19]:
output_fits = fits.PrimaryHDU(gaussian_data)
output_fits.writeto('/home/scratch/hfwest/Pilot/Results/gaussian_map_11.fits', overwrite=True)